<a href="https://colab.research.google.com/github/Odai1725873/MV-Project/blob/main/Face_Recognition_With_Voice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Open Camera**


In [ ]:
import cv2
import torch

# Load the model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Add the missing definitions for inputs and labels
inputs = ... # Add code here to initialize inputs with data
labels = ... # Add code here to initialize labels with data
inputs, labels = inputs.to(device), labels.to(device)

checkpoint = torch.load(r"best_.pt", map_location=torch.device('cpu'))
model = checkpoint['model']
model.eval()

# Open the webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Pre-process the frame
    frame = frame.transpose((2, 0, 1)) # Transpose the input so that it is in the format (C, H, W)
    frame = torch.FloatTensor(frame).to(torch.device('cpu')) # Convert the input to a Tensor
    frame = frame.unsqueeze(0) # Adds a batch dimension to the input

    # Run the model on the frame
    with torch.no_grad():
        output = model(frame)

    # Post-process the output
    # TODO: Add code here to post-process the output

    # Display the result
    frame = frame.squeeze().transpose((1, 2, 0)) # Transpose the output back to its original format (H, W, C)
    frame = frame.numpy().astype('uint8') # Convert the Tensor back to a numpy array
    cv2.imshow("Webcam", frame)
    if cv2.waitKey(1) == 27:
        break

# Release the webcam
cap.release()
cv2.destroyAllWindows()

**Face recognition with voice**

In [ ]:
import cv2
import numpy as np
from os import listdir
from os.path import isfile,join
import serial
import time
import pyttsx3
q=1
x=0
c=0
m=0
d=0
while q<=2:
    data_path = 'C:/Users/user/Desktop/python/image/'
    onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path,f))]
    Training_data, Lebels = [],[]
    for i , files in enumerate(onlyfiles):
        image_path = data_path + onlyfiles[i]
        images = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        Training_data.append(np.asarray(images, dtype = np.uint8)) 
        Lebels.append(i)

    Lebels = np.asarray(Lebels, dtype = np.int32)
    model = cv2.face.LBPHFaceRecognizer_create()
    model.train(np.asarray(Training_data),np.asarray(Lebels))
    print("training complete")
    q+=1
face_classifier = cv2.CascadeClassifier('C:/Users/user/AppData/Local/Programs/Python/Python38\Lib/site-packages/cv2/data/haarcascade_frontalface_default.xml')


def speak(audio):
    engine.say(audio)
    engine.runAndWait()
engine = pyttsx3.init('sapi5')
voices=engine.getProperty('voices')
engine.setProperty("voice",voices[0].id)
engine.setProperty("rate",140)
engine.setProperty("volume",1000)


def face_detector(img, size= 0.5):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray,1.3,5)

    if faces is():
        return img,[]
    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi,(200,200))
    
    return img,roi

cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()

    image, face = face_detector(frame)

    try:
        face = cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        result= model.predict(face)
        if result[1]<500: